# ANOVA  - Lab

## Introduction

In this lab, you'll get some brief practice generating an ANOVA table (AOV) and interpreting its output. You'll also perform some investigations to compare the method to the t-tests you previously employed to conduct hypothesis testing.

## Objectives

In this lab you will: 

- Use ANOVA for testing multiple pairwise comparisons 
- Interpret results of an ANOVA and compare them to a t-test

In [17]:
# Libraries
import pandas as pd
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import ttest_ind

## Load the data

Start by loading in the data stored in the file `'ToothGrowth.csv'`: 

In [4]:
# Your code here
df = pd.read_csv("ToothGrowth.csv")
df.head()

,len,supp,dose
0,4.2,VC,0.5
1,11.5,VC,0.5
2,7.3,VC,0.5
3,5.8,VC,0.5
4,6.4,VC,0.5


## Generate the ANOVA table

Now generate an ANOVA table in order to analyze the influence of the medication and dosage:  

In [5]:
# Your code here
formula = 'len ~ C(supp)+ dose'
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

               sum_sq    df           F        PR(>F)
C(supp)    205.350000   1.0   11.446768  1.300662e-03
dose      2224.304298   1.0  123.988774  6.313519e-16
Residual  1022.555036  57.0         NaN           NaN


## Interpret the output

Make a brief comment regarding the statistics and the effect of supplement and dosage on tooth length: 

### Comment
Focusing on the probability column we can see that the p_values are less than alpha level(0.05).This indicates that both factors have a statistically significant effect on the response variable, and we can reject the null hypothesis for each. The two factors are all influential with categorical item supplement type less influential compared to dose,

## Compare to t-tests

Now that you've had a chance to generate an ANOVA table, its interesting to compare the results to those from the t-tests you were working with earlier. With that, start by breaking the data into two samples: those given the OJ supplement, and those given the VC supplement. Afterward, you'll conduct a t-test to compare the tooth length of these two different samples: 

In [10]:
# OJ df
df_OJ = df[df['supp']=='OJ']
df_OJ.head()

,len,supp,dose
30,15.2,OJ,0.5
31,21.5,OJ,0.5
32,17.6,OJ,0.5
33,9.7,OJ,0.5
34,14.5,OJ,0.5


In [11]:
# VC df
df_VC = df[df['supp']=='VC']
df_VC.head()

,len,supp,dose
0,4.2,VC,0.5
1,11.5,VC,0.5
2,7.3,VC,0.5
3,5.8,VC,0.5
4,6.4,VC,0.5


In [12]:
#checking shape 
print(df_OJ.shape)
print(df_VC.shape)

(30, 3)
(30, 3)


In [19]:
#creating samples
oj_sample = df_OJ['len']
vc_sample = df_VC['len']

Now run a t-test between these two groups and print the associated two-sided p-value: 

In [24]:
# Calculate the 2-sided p-value for a t-test comparing the two supplement groups
t_stat, p_value = ttest_ind(oj_sample,vc_sample, equal_var=True)
print("t-statistic:", t_stat)
print("two-sided p-value:", p_value)

if p_value < 0.05:
    print("Result: Significant difference between supplement groups (p < 0.05)")
else:
    print("Result: No significant difference between supplement groups (p ≥ 0.05)")


t-statistic: 1.91526826869527
two-sided p-value: 0.06039337122412849
Result: No significant difference between supplement groups (p ≥ 0.05)


## A 2-Category ANOVA F-test is equivalent to a 2-tailed t-test!

Now, recalculate an ANOVA F-test with only the supplement variable. An ANOVA F-test between two categories is the same as performing a 2-tailed t-test! So, the p-value in the table should be identical to your calculation above.

> Note: there may be a small fractional difference (>0.001) between the two values due to a rounding error between implementations. 

In [22]:
# Your code here; conduct an ANOVA F-test of the oj and vc supplement groups.
formula = 'len ~ C(supp)'
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)
# Compare the p-value to that of the t-test above. 
# They should match (there may be a tiny fractional difference due to rounding errors in varying implementations)

               sum_sq    df         F    PR(>F)
C(supp)    205.350000   1.0  3.668253  0.060393
Residual  3246.859333  58.0       NaN       NaN


The p_value for the 2 category ANOVA F-test is 0.060393 which equivalent with the two-sided p-value: 0.06039337122412849 of the t-test.

## Run multiple t-tests

While the 2-category ANOVA test is identical to a 2-tailed t-test, performing multiple t-tests leads to the multiple comparisons problem. To investigate this, look at the various sample groups you could create from the 2 features: 

In [30]:
group_name = []
group_dict = {}
for group in df.groupby(['supp', 'dose'])['len']:
    name = group[0]
    data = group[1]
    group_name.append(name)
    group_dict[name] = data

While bad practice, examine the effects of calculating multiple t-tests with the various combinations of these. To do this, generate all combinations of the above groups. For each pairwise combination, calculate the p-value of a 2-sided t-test. Print the group combinations and their associated p-value for the two-sided t-test.

In [ ]:
# Your code here; reuse your t-test code above to calculate the p-value for a 2-sided t-test
# for all combinations of the supplement-dose groups listed above. 
# (Since there isn't a control group, compare each group to every other group.)

In [31]:
import itertools
group_dict = {name: df[(df['supp'] == name[0]) & (df['dose'] == name[1])]['len'] for name in group_name}
combinations = list(itertools.combinations(group_name, 2))
print("Pairwise Welch's t-tests between (supp, dose) groups:\n")
for g1, g2 in combinations:
    sample1 = group_dict[g1]
    sample2 = group_dict[g2]
    
    t_stat, p_value = ttest_ind(sample1, sample2, equal_var=False) 
    
    print(f"{g1} vs {g2} --> p-value = {p_value:.5f}")

Pairwise Welch's t-tests between (supp, dose) groups:

('OJ', 0.5) vs ('OJ', 1.0) --> p-value = 0.00009
('OJ', 0.5) vs ('OJ', 2.0) --> p-value = 0.00000
('OJ', 0.5) vs ('VC', 0.5) --> p-value = 0.00636
('OJ', 0.5) vs ('VC', 1.0) --> p-value = 0.04601
('OJ', 0.5) vs ('VC', 2.0) --> p-value = 0.00001
('OJ', 1.0) vs ('OJ', 2.0) --> p-value = 0.03920
('OJ', 1.0) vs ('VC', 0.5) --> p-value = 0.00000
('OJ', 1.0) vs ('VC', 1.0) --> p-value = 0.00104
('OJ', 1.0) vs ('VC', 2.0) --> p-value = 0.09653
('OJ', 2.0) vs ('VC', 0.5) --> p-value = 0.00000
('OJ', 2.0) vs ('VC', 1.0) --> p-value = 0.00000
('OJ', 2.0) vs ('VC', 2.0) --> p-value = 0.96385
('VC', 0.5) vs ('VC', 1.0) --> p-value = 0.00000
('VC', 0.5) vs ('VC', 2.0) --> p-value = 0.00000
('VC', 1.0) vs ('VC', 2.0) --> p-value = 0.00009


## Summary

In this lesson, you implemented the ANOVA technique to generalize testing methods to multiple groups and factors.